<a href="https://colab.research.google.com/github/notDhruv/notDhruv.github.io/blob/master/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.utils.data as data

In [ ]:
# USE GPU(cuda) OR CPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [ ]:
#Number of images to be passed into the model in one iteration
batch_size = 64
#Auto download the MNIST dataset through Pytorch
train_dataset = datasets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor(),
                           download=True)
#Convert dataset to Pytorchs dataloader object so that we can directly get batches of images instead of single images
train_loader = data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Define the neural network model architecture in a class
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.model = nn.Sequential(
                    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
                    nn.AvgPool2d(kernel_size=2,stride=2),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
                    nn.AvgPool2d(kernel_size=2,stride=2),
                    nn.ReLU(),
                    nn.Conv2d(in_channels=16, out_channels=120, kernel_size=4, stride=1),
                    nn.ReLU(),
                    nn.Flatten(),
                    nn.Linear(in_features=120, out_features=32),
                    nn.ReLU(),
                    nn.Linear(32, 10),                    
        )
    def forward(self, input):
        output = self.model(input)
        return output

In [ ]:
# Create an object of the model class and then use to(device) is to move model to GPU if available
model = LeNet().to(device)

learning_rate = 1e-5
epochs = 25
num_train_images = len(train_dataset)
criterion = nn.CrossEntropyLoss()                                   # loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)        # Optimzer to perform the backpropogation

model = model.train()                                               # Set model to training mode(meaning you can update the weights)

In [ ]:
print(">>> Starting Training...")

for j in range(epochs):
    total_correct = 0
    for batch_num, (images, labels) in enumerate(train_loader):
        images = images.to(device)                  # Move to GPU if using GPU
        labels = labels.to(device)                  # Move to GPU if using GPU
        output = model(images)                      # Pass through the neural network model
        predicted_labels = torch.argmax(output, dim=1)  # The model outputs a 10 dimensional vector for each image in the batch. So output is 64 x 10. We reduce it to 64 x 1 by using argmax as other values arent of interest
        loss = criterion(output, labels)            # Calculate the loss value
        loss.backward()                             # Calculate the updates to be done to the model weights
        optimizer.step()                            # Make the updates to the model weights
        total_correct += torch.sum(labels==predicted_labels).item()         # how many of the predictions were correct in this batch
    accuracy = 100*(total_correct/num_train_images)
    print("Epoch %2d\t\tLoss %.4f\t\tAccuracy %.3f" % (j,loss.item(), accuracy))

>>> Starting Training...
Epoch  0		Loss 2.0249		Accuracy 33.385
Epoch  1		Loss 1.0380		Accuracy 59.247
Epoch  2		Loss 0.5370		Accuracy 74.790
Epoch  3		Loss 0.7044		Accuracy 80.597
Epoch  4		Loss 0.7015		Accuracy 81.997
Epoch  5		Loss 0.9177		Accuracy 81.625
Epoch  6		Loss 1.2139		Accuracy 82.968
Epoch  7		Loss 0.3299		Accuracy 84.782
Epoch  8		Loss 0.6070		Accuracy 86.005
Epoch  9		Loss 0.3161		Accuracy 87.193
Epoch 10		Loss 0.2789		Accuracy 88.027
Epoch 11		Loss 0.0959		Accuracy 88.903
Epoch 12		Loss 0.4438		Accuracy 89.800
Epoch 13		Loss 0.1168		Accuracy 90.602
Epoch 14		Loss 0.4646		Accuracy 91.258
Epoch 15		Loss 0.4106		Accuracy 91.747
Epoch 16		Loss 0.2298		Accuracy 92.160
Epoch 17		Loss 0.3641		Accuracy 92.587
Epoch 18		Loss 0.4675		Accuracy 92.978
Epoch 19		Loss 0.3177		Accuracy 93.365
Epoch 20		Loss 0.0960		Accuracy 93.622
Epoch 21		Loss 0.1552		Accuracy 93.952
Epoch 22		Loss 0.0622		Accuracy 94.237
Epoch 23		Loss 0.1983		Accuracy 94.488
Epoch 24		Loss 0.0688		Accuracy 94.768


In [ ]:
# Evaluate the model
model.eval()
num_test_images = len(test_dataset)
total_correct = 0
for batch_num, (images, labels) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    predicted_labels = torch.argmax(output, dim=1)
    total_correct += torch.sum(labels==predicted_labels).item()
accuracy = 100 * total_correct/num_test_images
print("Test Accuracy is %.3f" % (accuracy))

Test Accuracy is 94.960


In [ ]:
# Saving the model weights as a file to google drive
PATH = '/content/drive/My Drive/model_checkpoint.pt'
torch.save(model.state_dict(), PATH)

In [ ]:
# Reloading the model from the file in google drive
new_model = LeNet()
new_model.load_state_dict(torch.load(PATH))

<All keys matched successfully>